In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import re
from collections import deque
import datetime

## 1. 파일호출

In [4]:
# 파일 불러오기
with open('../../data/df_da.pkl', 'rb') as f:
    df_da = pickle.load(f)


In [5]:
# 파일 불러오기
with open('../../data/df_body.pkl', 'rb') as f:
    df_body = pickle.load(f)


In [6]:
df_da['date']   = df_da['q_creationdate'].dt.date
df_da['year']   = df_da['q_creationdate'].dt.year
df_da['month']  = df_da['q_creationdate'].dt.month
df_da['day']    = df_da['q_creationdate'].dt.day

In [7]:
df_body['date'] = df_body['creationdate'].dt.date
df_body['year'] = df_body['creationdate'].dt.year
df_body['month'] = df_body['creationdate'].dt.month
df_body['day'] = df_body['creationdate'].dt.day

In [8]:
df_body.head()

,creationdate,id,posttypeid,title,owneruserid,body,answered_yn,date,year,month,day
0,2023-10-26 15:34:53.830,77368423,2,None,508057.0,"<p>If all machines are on the same LAN, then y...",0,2023-10-26,2023,10,26
1,2023-11-01 08:52:42.577,77401389,1,Unable to do File Upload using sendkeys method...,22838353.0,<p>I am trying to automate a webpage using Sel...,0,2023-11-01,2023,11,1
2,2023-10-30 14:32:47.247,77389520,1,Run macro based on cell contents if cell changes,22828175.0,<p>I want to run a macro if a cell changes bas...,0,2023-10-30,2023,10,30
3,2023-10-30 14:36:20.843,77389551,2,None,12500315.0,<p>Whilst you <em>could</em> write your own wr...,0,2023-10-30,2023,10,30
4,2023-10-23 09:20:47.107,77344034,1,Asp.Net chunking cookie issue,3365923.0,<p>I am using ASP.NET Identity. The login app ...,0,2023-10-23,2023,10,23


## 2. Preprocessing 

In [9]:
# def cleanhtml(raw_html):
#   cleanr = re.compile('<(/)?([a-zA-Z0-9]*)(\\s[a-zA-Z0-9]*=[^>]*)?(\\s)*(/)?>')
#   cleantext = re.sub(cleanr, '', raw_html)
#   cleantext = re.sub(r"\n", "", cleantext)
#   cleantext = re.sub(r"\r", "", cleantext)
#   cleantext = re.sub(r"\t", "", cleantext)
#   return cleantext

In [10]:
def cleanhtml(raw_html):
  #1. replace newline to space
  # To analyze the text, it would be hard to understand all source code in python, so replace all the <code> tag first
  # src_re = re.compile('<code>(?=[^<]*\n)[\s\S]*?<\/code>')
  # src_re = re.compile('<code>[^<]*\\n[^<]*<\/code>')
  src_re = re.compile('<code>(.*?)<\/code>')
  cleantext_1 = re.sub(src_re, '', raw_html.replace('\n', '_**_'))
  cleantext_1 = cleantext_1.replace('_**_', '\n')

  tag_re = re.compile('<.*?>')
  cleantext_2 = re.sub(tag_re, '', cleantext_1)

  cleantext_2 = re.sub(r"\n", "", cleantext_2)
  cleantext_2 = re.sub(r"\r", "", cleantext_2)
  cleantext_2 = re.sub(r"\t", "", cleantext_2)

  return cleantext_2

In [11]:
# apply the function, cleanhtml to the question and body text
df_body['body_prep'] = df_body['body'].apply(cleanhtml)

In [12]:
df_wl = df_body[['id', 'creationdate', 'body_prep']].copy()

In [13]:
def wordlen(text):
    return [x for x in text.split(' ') if len(x) > 6]

In [14]:
df_wl['over6'] = df_wl['body_prep'].apply(wordlen)